In [131]:
import pandas as pd 
data_movies=pd.read_csv(r"C:\Users\veram\OneDrive\Escritorio\Data Science Henry\Proyecto individual I\movies_dataset.csv", encoding="utf-8", sep=',', na_values="NaN")

C:\Users\veram\AppData\Local\Temp\ipykernel_10520\39759258.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data_movies=pd.read_csv(r"C:\Users\veram\OneDrive\Escritorio\Data Science Henry\Proyecto individual I\movies_dataset.csv", encoding="utf-8", sep=',', na_values="NaN")


In [132]:
'''

revisar la data( tipos de datos, columnas, filas, memoria)

'''
data_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [133]:
''' 

Depuracion, limpieza y normalizacion de datos en una funcion

'''
def clean_data(data_movies):
    # Drop columns: 'adult', 'homepage' ,'status'and 10 other columns
    data_movies = data_movies.drop(columns=['adult', 'homepage', 'imdb_id', 'genres','status', 'poster_path', 'production_companies', 'production_countries', 'runtime', 'video', 'tagline', 'overview', 'original_title'])
    
# convertir texto : 'adult', 'belongs_to_collection' and 16 other columns

    data_movies['budget'] = data_movies['budget'].str.lower()
    data_movies['id'] = data_movies['id'].str.lower()
    data_movies['original_language'] = data_movies['original_language'].str.lower()
    data_movies['release_date'] = data_movies['release_date'].str.lower()
    data_movies['title'] = data_movies['title'].str.lower()
    
   
# borrar valores duplicados y nulos 
    data_movies = data_movies.dropna(subset=['title'])
    data_movies = data_movies.drop_duplicates(subset=['title'])

    
# filtar y eliminar  en la columna "vote average valores igual a 0", es un filtro para eliminar peliculas muy mal punteadas
    data_movies = data_movies[~(data_movies['vote_average'] == 0)]
    
# reemplazamos  missing values con "0" en 'revenue' y  'budget' para poder sacar el ROI
    data_movies = data_movies.fillna({'budget': "0"})
   
    data_movies = data_movies.fillna({'revenue': 0})
    

# cambiamos el tipo de dato de las columnas para standanrizar el tipo de dato y sean coorecto con las columnas 
    data_movies = data_movies.astype({'budget': 'string', 'id': 'string', 'popularity': 'string', 'revenue': 'string', 'title': 'string', 'vote_average': 'string', 'vote_count': 'string'})
    data_movies['release_date'] = pd.to_datetime(data_movies['release_date'], errors='coerce', format='%Y-%m-%d')
    data_movies = data_movies.astype({'budget': 'float64', 'popularity': 'float64', 'revenue': 'float64', 'vote_count': 'float64'})
    data_movies = data_movies.astype({'vote_count': 'int64', 'budget': 'int64', 'revenue': 'int64'})

# redondeamos el decimal en'popularity
    data_movies = data_movies.round({'popularity': 1})
    
    
    return data_movies
data_movies_clean = clean_data(data_movies)


In [134]:
#eliminamos las filas con doble nivel

def eliminar_filas_doble_nivel(data_movies_clean):
    # Verificar si el DataFrame tiene columnas de doble nivel
    if data_movies_clean.columns.nlevels > 1:
        # Encontrar filas que contengan datos en columnas de doble nivel
        filas_con_doble_nivel = data_movies_clean[data_movies_clean.apply(lambda x: x.index.nlevels > 1, axis=1)].index
        
        # Eliminar las filas identificadas
        data_movies_clean = data_movies_clean.drop(filas_con_doble_nivel)
    
    return data_movies_clean

df_limpiado = eliminar_filas_doble_nivel(data_movies_clean) # ¿llamado de la funcion

In [135]:
'''renombramos las columnas de nuestro datasframe '''

def rename_data(df_limpiado):
  
    # Renombrar las columnas según el diccionario dado
    df_limpiado = df_limpiado.rename(columns={
        'budget': 'presupuesto',
        'id': 'id_pelicula',
        'release_date': 'fecha_estreno',
        'revenue': 'ganancias_ingresos',
        'title': 'titulo_pelicula'
    })
    return df_limpiado

movies_clean = rename_data(df_limpiado) #llamado a la funcion

In [136]:
''' Función para calcular ROI (Return on Investment)'''

def calcular_roi(row):
    ganancias = row['ganancias_ingresos']
    presupuesto = row['presupuesto']
    if presupuesto != 0:
        roi = (ganancias - presupuesto) / presupuesto
        roi = round(roi, 2)
    else:
        roi = 0  # Manejar el caso donde el presupuesto es cero
    
    return roi

# Aplicar la función al DataFrame y crear una nueva columna 'ROI'
movies_clean['ROI'] = movies_clean.apply(calcular_roi, axis=1)


In [137]:
'''creamos nuevas columas con el año, mes y dia en base a nuestra fecha de estreno'''

movies_clean['year_estreno'] =movies_clean['fecha_estreno'].dt.year
movies_clean['mes_estreno'] =movies_clean['fecha_estreno'].dt.month
movies_clean['dia_estreno'] =movies_clean['fecha_estreno'].dt.day

print(movies_clean)  #revisamos el dataset 

                                   belongs_to_collection  presupuesto  \
0      {'id': 10194, 'name': 'Toy Story Collection', ...     30000000   
1                                                    NaN     65000000   
2      {'id': 119050, 'name': 'Grumpy Old Men Collect...            0   
3                                                    NaN     16000000   
4      {'id': 96871, 'name': 'Father of the Bride Col...            0   
...                                                  ...          ...   
45457                                                NaN            0   
45458                                                NaN            0   
45459                                                NaN            0   
45461                                                NaN            0   
45462                                                NaN            0   

      id_pelicula original_language  popularity fecha_estreno  \
0             862                en        21.9    1995-10

In [138]:
#para poder mapear los dias, meses y años hay que convertir a entero 

movies_clean['year_estreno'] =movies_clean['year_estreno'].fillna(0).astype(int)
movies_clean['mes_estreno'] = movies_clean['mes_estreno'].fillna(0).astype(int)
movies_clean['dia_estreno'] = movies_clean['dia_estreno'].fillna(0).astype(int)

movies_clean.head() # revisamos el dataset

,belongs_to_collection,presupuesto,id_pelicula,original_language,popularity,fecha_estreno,ganancias_ingresos,spoken_languages,titulo_pelicula,vote_average,vote_count,ROI,year_estreno,mes_estreno,dia_estreno
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,862,en,21.9,1995-10-30,373554033,"[{'iso_639_1': 'en', 'name': 'English'}]",toy story,7.7,5415,11.45,1995,10,30
1,NaN,65000000,8844,en,17.0,1995-12-15,262797249,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",jumanji,6.9,2413,3.04,1995,12,15
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,15602,en,11.7,1995-12-22,0,"[{'iso_639_1': 'en', 'name': 'English'}]",grumpier old men,6.5,92,0.00,1995,12,22
3,NaN,16000000,31357,en,3.9,1995-12-22,81452156,"[{'iso_639_1': 'en', 'name': 'English'}]",waiting to exhale,6.1,34,4.09,1995,12,22
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0,11862,en,8.4,1995-02-10,76578911,"[{'iso_639_1': 'en', 'name': 'English'}]",father of the bride part ii,5.7,173,0.00,1995,2,10


In [139]:
#hacemos una lista de los dias que contiene un mes y asignamos nombre

dia_letra = {1: 'lunes',2: 'martes',3: 'miercoles',4: 'jueves',5: 'viernes',6: 'sabado',7: 'domingo',8: 'lunes',9: 'martes',10: 'miercoles',
             11: 'jueves',12: 'viernes',13: 'sabado',14: 'domingo',15: 'lunes',16: 'martes',17: 'miercoles',18: 'jueves',19: 'viernes',20: 'sabado',
             21: 'domingo',22: 'lunes',23: 'martes',24: 'miercoles',25: 'jueves',26: 'viernes',27: 'sabado',28: 'domingo',29: 'lunes',30: 'martes',31: 'miercoles'
            }

# Aplicar el mapeo a la columna "dias"
movies_clean['dia_estreno'] = movies_clean['dia_estreno'].map(dia_letra)

print(movies_clean) # Mostrar el DataFrame resultante



                                   belongs_to_collection  presupuesto  \
0      {'id': 10194, 'name': 'Toy Story Collection', ...     30000000   
1                                                    NaN     65000000   
2      {'id': 119050, 'name': 'Grumpy Old Men Collect...            0   
3                                                    NaN     16000000   
4      {'id': 96871, 'name': 'Father of the Bride Col...            0   
...                                                  ...          ...   
45457                                                NaN            0   
45458                                                NaN            0   
45459                                                NaN            0   
45461                                                NaN            0   
45462                                                NaN            0   

      id_pelicula original_language  popularity fecha_estreno  \
0             862                en        21.9    1995-10

In [140]:
#creamos una lista con los meses del año y asignamos nombre para poder mapearlos en la columna

mes_letra = {1: 'enero',2: 'febrero',3: 'marzo',4: 'abril',5: 'mayo',6: 'junio',7: 'julio',8: 'agosto',9: 'septiembre',10: 'octubre',
             11: 'noviembre',12: 'diciembre'
            }

# Aplicar el mapeo a la columna 'meses'
movies_clean['mes_estreno'] = movies_clean['mes_estreno'].map(mes_letra)

print(movies_clean) # Mostrar el DataFrame resultante

                                   belongs_to_collection  presupuesto  \
0      {'id': 10194, 'name': 'Toy Story Collection', ...     30000000   
1                                                    NaN     65000000   
2      {'id': 119050, 'name': 'Grumpy Old Men Collect...            0   
3                                                    NaN     16000000   
4      {'id': 96871, 'name': 'Father of the Bride Col...            0   
...                                                  ...          ...   
45457                                                NaN            0   
45458                                                NaN            0   
45459                                                NaN            0   
45461                                                NaN            0   
45462                                                NaN            0   

      id_pelicula original_language  popularity fecha_estreno  \
0             862                en        21.9    1995-10

In [141]:
"""
Despues de los cambios y una nueva revision al dataser borramos otras columnas que no son de interes para nuestro proyecto
"""
def clean_data(movies_clean):
    
    movies_clean = movies_clean.drop(columns=['belongs_to_collection', 'fecha_estreno', 'spoken_languages'])
    return movies_clean

movies_clean_1 = clean_data(movies_clean.copy())
movies_clean_1.head()

,presupuesto,id_pelicula,original_language,popularity,ganancias_ingresos,titulo_pelicula,vote_average,vote_count,ROI,year_estreno,mes_estreno,dia_estreno
0,30000000,862,en,21.9,373554033,toy story,7.7,5415,11.45,1995,octubre,martes
1,65000000,8844,en,17.0,262797249,jumanji,6.9,2413,3.04,1995,diciembre,lunes
2,0,15602,en,11.7,0,grumpier old men,6.5,92,0.00,1995,diciembre,lunes
3,16000000,31357,en,3.9,81452156,waiting to exhale,6.1,34,4.09,1995,diciembre,lunes
4,0,11862,en,8.4,76578911,father of the bride part ii,5.7,173,0.00,1995,febrero,miercoles


In [142]:
#limpiamos nuestra columna del titulo de caracteres "extraños" para filtrar mejor la data
pattern = r'[^a-zA-Z0-9,:\'" ]'
filtered_movies_clean = movies_clean_1[~movies_clean_1['titulo_pelicula'].str.contains(pattern, regex=True)]

In [143]:
# Eliminar filas con valores NaN o nulos en la data obtenida
filtered_movies_clean = filtered_movies_clean.dropna(subset=['ganancias_ingresos', 'presupuesto', 'titulo_pelicula','year_estreno','mes_estreno','dia_estreno' ])

In [144]:
'''Al abrir la data nos encontramos con varias columnas anidadas 
 solo una resultaba de interes para desarrollar el enfoque del proyecto, la separamos y tratamos por aparte
 la unimos a la data depurada por medio del id de las peliculas'''

'Al abrir la data nos encontramos con varias columnas anidadas \n solo una resultaba de interes para desarrollar el enfoque del proyecto, la separamos y tratamos por aparte\n la unimos a la data depurada por medio del id de las peliculas'

In [145]:
generos=pd.read_csv(r"C:\Users\veram\OneDrive\Escritorio\Data Science Henry\Proyecto individual I\ETL\Generos_.csv")

In [146]:
generos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79794 entries, 0 to 79793
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   generos  79794 non-null  object
dtypes: object(1)
memory usage: 623.5+ KB


In [147]:
data = pd.concat([generos, filtered_movies_clean], axis=1, ignore_index=True)

In [148]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79794 entries, 0 to 79793
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       79794 non-null  object 
 1   1       35870 non-null  float64
 2   2       35870 non-null  string 
 3   3       35864 non-null  object 
 4   4       35870 non-null  float64
 5   5       35870 non-null  float64
 6   6       35870 non-null  string 
 7   7       35870 non-null  string 
 8   8       35870 non-null  float64
 9   9       35870 non-null  float64
 10  10      35870 non-null  float64
 11  11      35870 non-null  object 
 12  12      35870 non-null  object 
dtypes: float64(6), object(4), string(3)
memory usage: 7.9+ MB


In [149]:
"""
la funcion con el tratamiento de los datos luego de concatenarlos
"""
def clean_data(data):
#eliminamos valores nulos y repetidos por medio filtrando la columna de titulos
    data = data.drop_duplicates(subset=[6])
    data = data.dropna()
#Capitalizamos el primer caracter de las columnas con strings
    data[0] = data[0].str.capitalize()
    data[3] = data[3].str.capitalize()
    data[6] = data[6].str.capitalize()
    data[11] = data[11].str.capitalize()
    data[12] = data[12].str.capitalize()
   
   
#renombrar las columnas, perdieron su nombre original con la concatenacion    
    data = data.rename(columns={0: 'generos'})
    # Rename column 12 to 'dia'
    data = data.rename(columns={12: 'dia'})
    # Rename column 11 to 'mes'
    data = data.rename(columns={11: 'mes'})
    # Rename column 10 to 'year'
    data = data.rename(columns={10: 'year'})
    # Rename column 6 to 'titulo_pelicula'
    data = data.rename(columns={6: 'titulo_pelicula'})
    # Rename column 7 to 'vote_average'
    data = data.rename(columns={7: 'vote_average'})
    # Rename column 3 to 'idioma'
    data = data.rename(columns={3: 'idioma'})
    # Rename column 1 to 'presupuesto'
    data = data.rename(columns={1: 'presupuesto'})
    # Rename column 4 to 'popularidad'
    data = data.rename(columns={4: 'popularidad'})
    # Rename column 5 to 'ganancias'
    data = data.rename(columns={5: 'ganancias'})
    # Rename column 8 to 'vote_count'
    data = data.rename(columns={8: 'vote_count'})
    # Rename column 9 to 'ROI'
    data = data.rename(columns={9: 'ROI'})
#luego de otra revicion decidimos eliminar otra innecesaria para el proyecto
    data = data.drop(columns=[2])
    return data

data_completa = clean_data(data.copy())  # llamado de la funcion

In [152]:
columnas_a_entero = ['ROI','ganancias','presupuesto','year','vote_count']
data_completa[columnas_a_entero] = data_completa[columnas_a_entero].astype(int)

In [153]:

data_completa

,generos,presupuesto,idioma,popularidad,ganancias,titulo_pelicula,vote_average,vote_count,ROI,year,mes,dia
0,Animation,30000000,En,21.9,373554033,Toy story,7.7,5415,11,1995,Octubre,Martes
1,Adventure,65000000,En,17.0,262797249,Jumanji,6.9,2413,3,1995,Diciembre,Lunes
2,Romance,0,En,11.7,0,Grumpier old men,6.5,92,0,1995,Diciembre,Lunes
3,Comedy,16000000,En,3.9,81452156,Waiting to exhale,6.1,34,4,1995,Diciembre,Lunes
4,Comedy,0,En,8.4,76578911,Father of the bride part ii,5.7,173,0,1995,Febrero,Miercoles
...,...,...,...,...,...,...,...,...,...,...,...,...
45456,Romance,0,En,0.2,0,House of horrors,6.3,8,0,1946,Marzo,Lunes
45457,Drama,0,En,0.1,0,Shadow of the blair witch,7.0,2,0,2000,Octubre,Lunes
45458,Romance,0,En,0.4,0,The burkittsville 7,7.0,1,0,2000,Octubre,Miercoles
45459,Comedy,0,En,0.7,0,Caged heat 3000,3.5,1,0,1995,Enero,Lunes
